In [48]:
from imblearn.over_sampling import SMOTE
import cv2 as cv
import numpy as np
import os
from collections import defaultdict

In [125]:
!gdown "https://drive.google.com/uc?id=1NZhqBL0DFYHCUBaqSGzic7wE3P-XZvtU"

Error:

	HTTPSConnectionPool(host='drive.google.com', port=443): Max retries
	exceeded with url: /uc?id=1NZhqBL0DFYHCUBaqSGzic7wE3P-XZvtU (Caused by
	NameResolutionError("<urllib3.connection.HTTPSConnection object at
	0x104403790>: Failed to resolve 'drive.google.com' ([Errno 8] nodename
	nor servname provided, or not known)"))

To report issues, please visit https://github.com/wkentaro/gdown/issues.


In [ ]:
!unzip custom_dataset_1.zip

In [93]:
parentDir = "custom_dataset/train/"
imageNames = []
labelNames = []

for file in os.listdir(parentDir + "images/"):
    imageNames.append(file)
    file = file[:-4]
    file += ".txt"
    labelNames.append(file)

In [94]:
labelNames[0]

'i-5375ae7c52114b15b01757b2fac4da65_jpg.rf.be3ad69a19172bdf88471a3aef44a5b8.txt'

In [95]:
# list to store all the cropped bounding boxes of all the images that are not bananas 
croppedBbs = defaultdict(list)

In [96]:
for label in labelNames:
   
    imgName = label[:-4] + ".jpg"
    imgPath = os.path.join(parentDir, "images", imgName)
    labelPath = os.path.join(parentDir, "labels", label)
    
    with open(labelPath, "r") as f:
        data = f.readlines()
        for line in data:
            values = line.split(sep=" ")
            values = list(map(lambda x: float(x), values))
            # if it is not a banana, add it to the dict
            if values[0] != 1:
                
                # xCentre, yCentre, width, height
                cords = values[1:]
                
                img = cv.imread(imgPath)
                wt = img.shape[1]
                ht = img.shape[0]
                
                # bounding box
                xCentre = float(cords[0]) * wt
                yCentre = float(cords[1]) * ht
                width = float(cords[2]) * wt
                height = float(cords[3]) * ht


                xmin = int(xCentre - width//2)
                xmax = int(xCentre + width//2)
                ymin = int(yCentre - height//2)
                ymax = int(yCentre + height//2)
                
                img = img[ymin : ymax, xmin : xmax]
                
                
                # a tuple that has : cropped img, box coords in original image, og image name
                croppedBbs[values[0]].append((img, cords, imgPath))
            # print(values)
        

In [124]:
# store all the cropped images 
imgList = []
# store tuples of the bounding box cords and og image name, as the "labels"
labelsList = []

for classNo in croppedBbs:
    
    imgTuples = croppedBbs[classNo]
    
    for tups in imgTuples:
        img, cords, imgPath = tups
        img = img.flatten()
        imgList.append(img)
        labelsList.append((cords, imgPath))
    
    
       
        # print(tups)

In [123]:
sm = SMOTE(random_state=33)
imgListNew, LabelsListNew = sm.fit_resample(imgList, labelsList)


TypeError: '<' not supported between instances of 'str' and 'list'